# 한빛 아카데미 - 새로나온 책 10page
* https://www.hanbit.co.kr/academy/books/new_book_list.html
* 제목 / 저자 / 번역 / 출간일 / 쪽수 / 판매가

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

# 1. 찾으려고 하는 목록 데이터

In [4]:
lis = soup.select('li.sub_book_list')
len(lis)

20

##  하나의 데이터로 원하는 정보를 추출

In [5]:
li = lis[0]

In [6]:
# 서브 페이지의 링크 추출
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B5620881464'

In [7]:
# sub page로 이동해서 원하는 정보를 추출
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
sub_soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>STEM CookBook, 전기 및 하이브리드 자동차(3판)</title>
<link href="https://www.hanbit.co.kr/images/common/hanbit.ico" rel="shortcut icon"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="STEM CookBook, 전기 및 하이브리드 자동차(3판)" property="og:title"/>
<meta content="이 책은 친환경자동차 중 가장 보편적인 전기 차량(EV)과 하이브리드 차량(HEV)에 대한 전반적인 개념과 원리를 소개한다." property="og:description"/>
<meta content="https://www.hanbit.co.kr/data/books/B5620881464_m.jpg" property="og:image"/>
<meta content="https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B5620881464" property="og:url"/>
<link href="https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B5620881464" rel="canonical">
<meta content="전기자동차,하이브리드 자동차,친환경자동차,EV,HEV,ICEV,모터,발전기,배터리,전기 기계" name="keywords"/>
<meta content="이 책은 

In [11]:
# title
title = sub_soup.select_one('.store_product_info_box > h3').text.strip()
title

'STEM CookBook, 전기 및 하이브리드 자동차(3판)'

In [12]:
info_list = sub_soup.select('.info_list > li')
info_list

[<li><strong>저자 : </strong><span>Iqbal Husain </span></li>,
 <li><strong>번역 : </strong><span>임원식 , 김정윤 , 김기찬 , 강동우 </span></li>,
 <li><strong>출간 : </strong><span>2023-01-10</span></li>,
 <li><strong>페이지 : </strong><span>592 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791156646372</span></li>,
 <li><strong>물류코드 :</strong><span>4637</span></li>]

In [16]:
info_dict = {}
for li in info_list:
    key = li.find('strong').text.split(' ')[0]
    value = li.find('span').text.strip()
    info_dict[key] = value
info_dict

{'저자': 'Iqbal Husain',
 '번역': '임원식 , 김정윤 , 김기찬 , 강동우',
 '출간': '2023-01-10',
 '페이지': '592 쪽',
 'ISBN': '9791156646372',
 '물류코드': '4637'}

In [18]:
author = info_dict['저자']
translator = info_dict['번역']
date = info_dict['출간']
page = info_dict['페이지']

author, translator, date, page

('Iqbal Husain', '임원식 , 김정윤 , 김기찬 , 강동우', '2023-01-10', '592 쪽')

In [19]:
## 번역이 없는 경우
try:
    translator = info_dict['번역']
except:
    translator = '-'

# 2. 한 페이지 정보 가져오기

In [25]:
lines=[]
for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
    
    title = sub_soup.select_one('.store_product_info_box > h3').text.strip()
    
    info_list = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_list:
        key = li.find('strong').text.split(' ')[0]
        value = li.find('span').text.strip()
        info_dict[key] = value
        info_dict
    
    author = info_dict['저자']
    try:
        translator = info_dict['번역']
    except:
        translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
    price = int(sub_soup.select_one('.pbr > strong').text.replace(',',''))
    lines.append({'제목' : title, '저자' : author, '번역' : translator, '출간일' : date, '쪽수' : page, '판매가' : price})

In [26]:
lines

[{'제목': 'STEM CookBook, 전기 및 하이브리드 자동차(3판)',
  '저자': 'Iqbal Husain',
  '번역': '임원식 , 김정윤 , 김기찬 , 강동우',
  '출간일': '2023-01-10',
  '쪽수': 592,
  '판매가': 35000},
 {'제목': 'IT CookBook, 컴퓨터 비전과 딥러닝',
  '저자': '오일석',
  '번역': '-',
  '출간일': '2023-01-05',
  '쪽수': 664,
  '판매가': 39000},
 {'제목': '리더십 : 이론과 실제 (9판)',
  '저자': 'Peter G. Northouse',
  '번역': '김남현',
  '출간일': '2023-01-02',
  '쪽수': 660,
  '판매가': 36000},
 {'제목': 'IT CookBook, JAVA 마스터',
  '저자': '송미영',
  '번역': '-',
  '출간일': '2023-01-01',
  '쪽수': 732,
  '판매가': 30000},
 {'제목': 'IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)',
  '저자': '최성철',
  '번역': '-',
  '출간일': '2023-01-01',
  '쪽수': 540,
  '판매가': 30000},
 {'제목': 'IT CookBook, 생활 속 예제로 배우는 생생 데이터 분석',
  '저자': '조인석',
  '번역': '-',
  '출간일': '2023-01-01',
  '쪽수': 428,
  '판매가': 23000},
 {'제목': 'IT CookBook, 파워포인트 2019',
  '저자': '김지연',
  '번역': '-',
  '출간일': '2023-01-01',
  '쪽수': 440,
  '판매가': 20000},
 {'제목': '메타버스로 구현하는 나만의 세상',
  '저자': '한규정 , 최종원 , 강신조 , 심민정 , 안형준 , 위새람 , 최동일',
  '번역': '-',
  '출간일': '2023-01-01

## DataFrame으로 변환

In [30]:
df = pd.DataFrame(lines)
df.head()

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 전기 및 하이브리드 자동차(3판)",Iqbal Husain,"임원식 , 김정윤 , 김기찬 , 강동우",2023-01-10,592,35000
1,"IT CookBook, 컴퓨터 비전과 딥러닝",오일석,-,2023-01-05,664,39000
2,리더십 : 이론과 실제 (9판),Peter G. Northouse,김남현,2023-01-02,660,36000
3,"IT CookBook, JAVA 마스터",송미영,-,2023-01-01,732,30000
4,"IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)",최성철,-,2023-01-01,540,30000


# 3. 10페이지 정보 가져오기

In [31]:
lines = []
for page in tqdm(range(1,11)):
    url = base_url + sub_url + '?page=' + str(page)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select('li.sub_book_list')
    
    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').text.strip()

        info_list = sub_soup.select('.info_list > li')
        info_dict = {}
        for li in info_list:
            key = li.find('strong').text.split(' ')[0]
            value = li.find('span').text.strip()
            info_dict[key] = value
            info_dict

        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        pages = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
        price = int(sub_soup.select_one('.pbr > strong').text.replace(',',''))
        lines.append({'제목' : title, '저자' : author, '번역' : translator, '출간일' : date, '쪽수' : pages, '판매가' : price})

  0%|          | 0/10 [00:00<?, ?it/s]

In [32]:
df = pd.DataFrame(lines)
df

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 전기 및 하이브리드 자동차(3판)",Iqbal Husain,"임원식 , 김정윤 , 김기찬 , 강동우",2023-01-10,592,35000
1,"IT CookBook, 컴퓨터 비전과 딥러닝",오일석,-,2023-01-05,664,39000
2,리더십 : 이론과 실제 (9판),Peter G. Northouse,김남현,2023-01-02,660,36000
3,"IT CookBook, JAVA 마스터",송미영,-,2023-01-01,732,30000
4,"IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)",최성철,-,2023-01-01,540,30000
...,...,...,...,...,...,...
195,NO BULLSHIT 선형대수학 가이드,Ivan Savov,"김대정 , 모현선 , 배준우",2020-01-06,628,28000
196,NO BULLSHIT 수학&물리 가이드,Ivan Savov,권기영,2020-01-06,568,28000
197,정보교과교육론(2판),"이태욱 , 최현종 , 전용주",-,2020-01-06,464,27000
198,실무로 완성하는 PLC(멜섹Q) : 서보를 이용한 위치 제어,정완보,-,2020-01-02,568,28000


In [33]:
df.to_csv('hanbitNewbook.csv', index=False)